In [1]:
from birdy import WPSClient
import birdy 

finch_server = 'http://localhost:8092/wps'   # finch 

finch_i = WPSClient(url=finch_server, progress=True)
finch = WPSClient(finch_server)

In [28]:
from os import path, listdir
from os.path import basename 

path = '/tmp/data/AFR-22/pr/'
url = 'http://localhost:8092/outputs/data/AFR-22/pr/'

# lists all files stored in 'path'
files = listdir(path)       

# sort files creating the urls for each file.
ds = [url + f for f in files ] 
# print some examples 
print(ds[0:5])

[]


In [19]:
# pf? for general exploration on processes provided by flyingpigeon
# help(finch_i.subset_bbox)

In [23]:
# similar example with finchs bbox subset:

lon0 = 0 # Minimum longitude.
lon1 = 17.0 # Maximum longitude.
lat0 = 10.5 # Minimum latitude.
lat1 = 24.0 # Maximum latitude.

out = finch_i.subset_bbox(
    resource=ds[0:5], lon0=lon0, lon1=lon1, lat0=lat0, lat1=lat1)

 owslib.wps.WPSException : {'code': 'NoApplicableCode', 'locator': 'None', 'text': "Process error: method=wps_xsubsetbbox.py._handler, line=170, msg='time_bnds' has more than 1-dimension and the same name as one of its dimensions ('time', 'time_bnds'). xarray disallows such variables because they conflict with the coordinates used to label dimensions."}


In [30]:
from metalink import download
tmp_out = '/home/nils/nap_niger/data/tmp/'

bbox_files = download.get(out.get()[1], path=tmp_out, segmented=False)

Metalink content-type detected.

Already downloaded pr_AFR-22_MOHC-HadGEM2-ES_rcp26_r1i1p1_GERICS-REMO2015_v1_day_20410101-20451230_sub.nc.

Already downloaded pr_AFR-22_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day_20060101-20101231_sub.nc.

Already downloaded pr_AFR-22_ECMWF-ERAINT_evaluation_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day_19790101-19801231_sub.nc.

Already downloaded pr_AFR-22_ECMWF-ERAINT_evaluation_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day_19810101-19851231_sub.nc.

Already downloaded pr_AFR-22_ECMWF-ERAINT_evaluation_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day_19860101-19901231_sub.nc.

Already downloaded pr_AFR-22_ECMWF-ERAINT_evaluation_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day_19910101-19951231_sub.nc.

Already downloaded pr_AFR-22_ECMWF-ERAINT_evaluation_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day_19960101-20001231_sub.nc.

Already downloaded pr_AFR-22_ECMWF-ERAINT_evaluation_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day_20010101-20051231_sub.nc.

Already downloaded pr_AFR-22_ECMWF-ERAINT_evalua

In [32]:
bbox_files = [tmp_out + f for f in listdir(tmp_out)]
print(len(bbox_files))

151


In [33]:
# bbox_ds = sort_by_filename(bbox_files)

bbox_ds = {}
for file in bbox_files:
    ds_name = basename(file)[:-25]
    if not ds_name in bbox_ds.keys():
        bbox_ds[ds_name] = [f for f in bbox_files if ds_name in f]
print(bbox_ds.keys())

dict_keys(['pr_AFR-22_MOHC-HadGEM2-ES_rcp26_r1i1p1_GERICS-REMO2015_v1_day', 'pr_AFR-22_MOHC-HadGEM2-ES_historical_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day', 'pr_AFR-22_MOHC-HadGEM2-ES_rcp85_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day', 'pr_AFR-22_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day', 'pr_AFR-22_MOHC-HadGEM2-ES_rcp26_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day', 'pr_AFR-22_NCC-NorESM1-M_historical_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day', 'pr_AFR-22_MOHC-HadGEM2-ES_rcp85_r1i1p1_GERICS-REMO2015_v1_day', 'pr_AFR-22_ECMWF-ERAINT_evaluation_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day', 'pr_AFR-22_ECMWF-ERAINT_evaluation_r1i1p1_GERICS-REMO2015_v1_day', 'pr_AFR-22_MPI-M-MPI-ESM-LR_historical_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day', 'pr_AFR-22_MOHC-HadGEM2-ES_historical_r1i1p1_GERICS-REMO2015_v1_day'])


In [34]:
bbox_ds['pr_AFR-22_ECMWF-ERAINT_evaluation_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day'][0]

'/home/nils/nap_niger/data/tmp/pr_AFR-22_ECMWF-ERAINT_evaluation_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day_20160101-20161231_sub.nc'

In [35]:
# systemcall to merge files belonging to one dataset
# ncrcat -h infiles outfile

from os import system
bbox_out = '/home/nils/nap_niger/data/bbox/'

for key in bbox_ds.keys():
    print(key)
    bbox_ds[key].sort()
    start = bbox_ds[key][0].split('_')[-2].split('-')[0]
    end = bbox_ds[key][-1].split('_')[-2].split('-')[1]
    outfile = bbox_out + key + '_' + start + '-' + end + '.nc'
    cmd = 'ncrcat -h {}{}* {}'.format(tmp_out, key, outfile )
    system(cmd)
    print('merged for {}'.format(outfile))

pr_AFR-22_MOHC-HadGEM2-ES_rcp26_r1i1p1_GERICS-REMO2015_v1_day
merged for /home/nils/nap_niger/data/bbox/pr_AFR-22_MOHC-HadGEM2-ES_rcp26_r1i1p1_GERICS-REMO2015_v1_day_20060101-20991230.nc
pr_AFR-22_MOHC-HadGEM2-ES_historical_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day
merged for /home/nils/nap_niger/data/bbox/pr_AFR-22_MOHC-HadGEM2-ES_historical_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day_19500101-20051230.nc
pr_AFR-22_MOHC-HadGEM2-ES_rcp85_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day
merged for /home/nils/nap_niger/data/bbox/pr_AFR-22_MOHC-HadGEM2-ES_rcp85_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day_20060101-20991230.nc
pr_AFR-22_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day
merged for /home/nils/nap_niger/data/bbox/pr_AFR-22_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day_20060101-21001231.nc
pr_AFR-22_MOHC-HadGEM2-ES_rcp26_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day
merged for /home/nils/nap_niger/data/bbox/pr_AFR-22_MOHC-HadGEM2-ES_rcp26_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day_20060101-20981230.nc
pr_AF

Based on the bbox subsets bias adjustmet had been manually performed.  
manually on the local mashine:

https://github.com/nilshempelmann/NAP_Niger/blob/master/bias_adjustment.R

**An appropriate WPS process will be included in birdhouse services soon!**


In [6]:
# After bias adjustment: 
# now they can be subsetted to the polygon of the country, process is available in flyingpigeon:

fp_server = 'http://localhost:8093/wps'    # flyingpigeon
fp_i = WPSClient(fp_server, progress=True)
fp = WPSClient(fp_server)

from os import path, listdir
path = '/home/nils/nap_niger/data/adjust/'

ds = [path+f for f in listdir(path)]
print(ds)

['/home/nils/nap_niger/data/adjust/tasAdjust_AFR-22_MOHC-HadGEM2-ES_historical_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day_19500101-20051230.nc']


In [7]:
# running flyingpigon polygonsubset file by file 
# out_all = []
# for n in range(len(names)):
#     resource = [url+f for f in files if names[n][:-20] in f]
#     for i in range(len(resource)):
        
#         out = fp_i.subset_countries(resource=resource[i], region='NER')
#         print(out.get()[1])
#         out_all.allocate(out.get()[1])

# print(out_all)

out = fp_i.subset_countries(resource=ds[0], region='NER')

In [9]:
out.get()

subset_countriesResponse(
    output='http://127.0.0.1:8093/outputs/f062c110-4748-11ea-bcfd-9cb6d08a53e7/tmp7seun2_t.tar',
    ncout='http://127.0.0.1:8093/outputs/f062c110-4748-11ea-bcfd-9cb6d08a53e7/tasAdjust_AFR-22_MOHC-HadGEM2-ES_historical_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day_19500101-20051230_NER.nc'
)

In [ ]:
# running flyingpigon polygonsubset file by file 
out_all = []
for n in range(len(names)):
    resource = [url+f for f in files if names[n][:-20] in f]
    for i in range(len(resource)):
        
        out = fp_i.subset_countries(resource=resource[i], region='NER')
        print(out.get()[1])
        out_all.allocate(out.get()[1])

print(out_all)

http://127.0.0.1:8093/outputs/ede764cc-3ea6-11ea-a34f-9cb6d08a53e7/pr_AFR-22_MOHC-HadGEM2-ES_rcp85_r1i1p1_GERICS-REMO2015_v1_day_20060101-20101230_NER.nc
http://127.0.0.1:8093/outputs/fb00f1dc-3ea6-11ea-85c2-9cb6d08a53e7/pr_AFR-22_MOHC-HadGEM2-ES_rcp85_r1i1p1_GERICS-REMO2015_v1_day_20110101-20151230_NER.nc
http://127.0.0.1:8093/outputs/07fef866-3ea7-11ea-8662-9cb6d08a53e7/pr_AFR-22_MOHC-HadGEM2-ES_rcp85_r1i1p1_GERICS-REMO2015_v1_day_20160101-20201230_NER.nc
http://127.0.0.1:8093/outputs/194c8426-3ea7-11ea-b7f6-9cb6d08a53e7/pr_AFR-22_MOHC-HadGEM2-ES_rcp85_r1i1p1_GERICS-REMO2015_v1_day_20210101-20251230_NER.nc
http://127.0.0.1:8093/outputs/28806f84-3ea7-11ea-9f80-9cb6d08a53e7/pr_AFR-22_MOHC-HadGEM2-ES_rcp85_r1i1p1_GERICS-REMO2015_v1_day_20260101-20301230_NER.nc
http://127.0.0.1:8093/outputs/38c4ab8a-3ea7-11ea-a7ac-9cb6d08a53e7/pr_AFR-22_MOHC-HadGEM2-ES_rcp85_r1i1p1_GERICS-REMO2015_v1_day_20310101-20351230_NER.nc
http://127.0.0.1:8093/outputs/47d2558c-3ea7-11ea-8adf-9cb6d08a53e7/pr_AFR-22

ReadTimeout: HTTPConnectionPool(host='localhost', port=8093): Read timed out. (read timeout=30)

In [8]:
## manual download to local file system ... (to be changed)

from os import path, listdir

path = '/home/nils/nap_niger/data/NER/'
# lists all files stored in 'path'
files = [ path+f for f in listdir(path) if 'tas_' in f]
files[:5]   # show the first files 

['/home/nils/nap_niger/data/NER/tas_AFR-22_MOHC-HadGEM2-ES_rcp85_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day_20410101-20451230_NER.nc',
 '/home/nils/nap_niger/data/NER/tas_AFR-22_MOHC-HadGEM2-ES_rcp85_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day_20160101-20201230_NER.nc',
 '/home/nils/nap_niger/data/NER/tas_AFR-22_MOHC-HadGEM2-ES_rcp26_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day_20110101-20151230_NER.nc',
 '/home/nils/nap_niger/data/NER/tas_AFR-22_MOHC-HadGEM2-ES_rcp26_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day_20760101-20801230_NER.nc',
 '/home/nils/nap_niger/data/NER/tas_AFR-22_ECMWF-ERAINT_evaluation_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day_19860101-19901231_NER.nc']

In [ ]:
# example file
f = tasAdjust_AFR-22_MOHC-HadGEM2-ES_historical_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_day_19500101-20051230_NER.nc'

In [30]:
def plot_file(resource): 
    from netCDF4 import Dataset, num2date
    from eggshell.nc.nc_utils import get_variable

    var = get_variable(resource)
    #print 'variable name: %s' % var
    ds = Dataset(resource)
    var = ds.variables[var]
    lat = ds.variables['lat']
    lon = ds.variables['lon']

    from matplotlib import pyplot as plt
    from cartopy import config
    from cartopy.util import add_cyclic_point
    import cartopy.crs as ccrs
    from numpy import meshgrid
    # to show the plots inline
    %matplotlib inline

    lons, lats = meshgrid(lon, lat)


    cs = plt.contourf(lons, lats, var[0,:,:])
    # plot first time stepp:
    ax = plt.axes(projection=ccrs.Robinson(central_longitude=0))
    ax.coastlines()
    ax.gridlines()
    cs = plt.contourf(lons, lats, var[0,:,:], 60, transform=ccrs.PlateCarree(), interpolation='nearest')
    #plt.colorbar()

#plt.clabel(cs, inline=1, fontsize=10)